In [7]:
import numpy as np
import snanapytools as snt
import os
from scipy.stats import norm, truncnorm, expon
import glob
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.constants import c 
__C_LIGHT_KMS__ = c.to('km/s').value
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.74, Om0=0.3089)
import os
plt.style.use(['seaborn-v0_8-deep', os.environ['HOME'] + '/.matplotlib_style/paper.mplstyle'])
import scipy.stats as scs
from pathlib import Path
import sys
sys.path.append('/global/homes/b/bastienc/')
sys.path.append('../scripts/')
import my_utils as mut
import paper_fun as pf
import flip

TRIPP_KEYS = ['alpha', 'beta', 'M_0', 'sigma_M', 'gamma']
models = {
    'P21': N, 
    'P21_NO_DUST', 'P21_FIXED_BETA', 'P21_FIXED_BETA_NO_DUST',
    'P21_REDUCED_TAU1', 'P21_REDUCED_TAU2', 'P21_REDUCED_TAU3',
    'P21_REDUCED_BETA1', 'P21_REDUCED_BETA2', 'P21_REDUCED_BETA3',
    'P21_REDUCED_TAU_BETA1', 'P21_REDUCED_TAU_BETA2', 'P21_REDUCED_TAU_BETA3'
}
PIPPIN_DIR = Path(os.environ['PIPPIN_OUTPUT'])

In [5]:
def beta_tau_modif(input_file, output_file, tau_scale=None, beta_scale=None):
    _format_dic = {
    'EBV': {'ZTRUE': '.2f', 'LOGMASS': '.2f'}, 
    'MAG_OFFSET': '.2f',
    'GENPEAK_SALT2BETA': '.8f',
    'GENSIGMA_SALT2BETA': '.8f'
    }

    if tau_scale is None and beta_scale is None:
        return
        
    par, doc = snt.tools.read_wgtmap(input_file)
    new_par = copy.deepcopy(par)
    if tau_scale is not None:
        prob = np.zeros(len(new_par['EBV']['EBV']))
        if tau_scale == 0:
            mask = new_par['EBV']['EBV'] == 0
            prob[mask] = 1
        else:
            mass_mask = new_par['EBV']['LOGMASS'] < 10

            plowM = scs.expon.pdf(new_par['EBV']['EBV'][mass_mask], scale=s * tau_lowM)
            plowM /= plowM.max()
            
            phighM = scs.expon.pdf(new_par['EBV']['EBV'][mass_mask], scale=s * tau_highM)
            phighM /= phighM.max()
            
            prob[mass_mask] = plowM
            prob[~mass_mask] = phighM
            
        new_par['EBV'].loc[:, 'PROB'] = prob 
    
    if beta_scale is not None:
        new_par['GENSIGMA_SALT2BETA'] = [s * new_par['GENSIGMA_SALT2BETA'][0], s * new_par['GENSIGMA_SALT2BETA'][1]]

    snt.tools.write_wgtmap(output_file,
                           new_par, docstr=doc, format_dic=format_dic)
      
    
        

In [6]:
file_origin = os.environ['SNDATA_ROOT'] + '/models/population_pdf/DES-SN5YR/DES-SN5YR_LOWZ_S3_P21.DAT.gz'

# VARIATION OF DIST
path = '/global/homes/b/bastienc/MY_SNANA_DIR/LSST_SNANA/DESCPub00196/pippin_files/snana_input/P23_variations/'
par, doc = snt.tools.read_wgtmap(file_1)
tau_lowM = 0.12
tau_highM = 0.15
format_dic = {
    'EBV': {'ZTRUE': '.2f', 'LOGMASS': '.2f'}, 
    'MAG_OFFSET': '.2f',
    'GENPEAK_SALT2BETA': '.8f',
    'GENSIGMA_SALT2BETA': '.8f'
    }

# NO DUST SET EBV = 0
beta_tau_modif(file_origin, 
               path  + "DES-SN5YR_LOWZ_S3_P21_NO_DUST.DAT", 
               tau_scale=0, beta_scale=None)

# BETA FIXED 
beta_tau_modif(file_origin, 
               path  + "DES-SN5YR_LOWZ_S3_P21_FIXED_BETA.DAT", 
               tau_scale=None, beta_scale=0)

# BETA FIXED + NO DUST
beta_tau_modif(file_origin, 
               path  + "DES-SN5YR_LOWZ_S3_P21_FIXED_BETA_NO_DUST.DAT", 
               tau_scale=0, beta_scale=0)

# REDUCED TAU
for s in [0.05, 0.25, 0.75]:
    beta_tau_modif(file_origin, 
                   path  + f"DES-SN5YR_LOWZ_S3_P21_REDUCED_TAU_{int(s)}_{int((s - int(s)) * 100):02d}.DAT", 
                   tau_scale=s, beta_scale=None)
# REDUCED BETA
for s in [0.05, 0.25, 0.75]:
    beta_tau_modif(file_origin, 
                   path  + f"DES-SN5YR_LOWZ_S3_P21_REDUCED_BETA_{int(s)}_{int((s - int(s)) * 100):02d}.DAT", 
                   tau_scale=None, beta_scale=s)

# REDUCED TAU & BETA
for s in [0.05, 0.25, 0.75]:
    beta_tau_modif(file_origin, 
                   path  + f"DES-SN5YR_LOWZ_S3_P21_REDUCED_TAU_BETA_{int(s)}_{int((s - int(s)) * 100):02d}.DAT", 
                   tau_scale=s, beta_scale=s)

In [8]:
fig, ax = plt.subplots(3,4,dpi=150, sharex=True, sharey=True, figsize=(10, 10))

files = Path('../pippin_files/snana_input/P23_variations/').glob('*')
    
    #snt.tools.read_wgtmap(f"DES-SN5YR_LOWZ_S3_P21_{model}")

# Beta * cint
mu_b, sig_b = 1.98, 0.35
dist_b = norm(mu_b, sig_b)
mu_c, sig_c = -0.084, 0.042
dist_c = norm(mu_c, sig_c)

# (Rv + 1) * EBV
mu_Rv = 2
sig_Rv = 1.4
min_Rv = 0.5
a_Rv = (min_Rv - mu_Rv) / sig_Rv
dist_Rv = truncnorm(a=a_Rv, b= np.inf, loc=mu_Rv, scale=sig_Rv)
tau_EBV = 0.17
dist_EBV = expon(scale=tau_EBV)

beta_salt = 3.12

N = 100000

beta =  dist_b.rvs(N)
c_int = dist_c.rvs(N)
betac = beta * c_int
Rv = dist_Rv.rvs(N)
EBV = dist_EBV.rvs(N)
RvEBV = (Rv + 1) * EBV
xth = np.linspace(-1, 1, 200)
plt.figure(figsize=(8, 6))

sns.kdeplot(betac, gridsize=300, bw_adjust=1.5, label='betac');
sns.kdeplot(RvEBV, gridsize=300, bw_adjust=1.5, label='RvEBV');
sns.kdeplot(betac+ RvEBV , gridsize=300, bw_adjust=1.5, label='tot');
sns.kdeplot(betac+ RvEBV + np.random.normal(scale=0.3, size=N), gridsize=300, bw_adjust=1.5, label='tot + noise');


plt.xlim(-1, 4)
plt.legend()
#sns.kdeplot(betac_noise, gridsize=100, bw_adjust=1.5);


IndentationError: unexpected indent (2043947547.py, line 5)

In [ ]:
fig, acs = plt.subplots()

In [ ]:
resfit = pd.read_csv('../results_p21_var/RES_MOCK00_P21_VAR_zrange_0.02_0.1_nojax.csv')
resfit = resfit[resfit.model == 'P21']
res = {}

del resfit

In [ ]:
MOCK_DIR_ORIGIN = PIPPIN_DIR / f'LSST_UCHUU_MOCK00_BC'
BBC_DIR_ORIGIN = MOCK_DIR_ORIGIN / '6_BIASCOR/LSST_P21/output'
BBC_FILE_ORIGIN = BBC_DIR_ORIGIN / f'OUTPUT_BBCFIT/FITOPT000_MUOPT000.FITRES.gz'

MOCK_DIR = PIPPIN_DIR / 'LSST_UCHUU_MOCK00_P21_VARIATIONS_BC'
FIT_DIR =  MOCK_DIR / '2_LCFIT'

zrange = 0.02, 0.1
# BBC DATA
df_BBC_ORIGIN = ascii.read(BBC_FILE_ORIGIN).to_pandas().set_index('CIDint')


In [ ]:
#####################
# COMPUTE USED CIDs #
#####################
# BBC DATA
df_BBC_ORIGIN = ascii.read(BBC_FILE_ORIGIN).to_pandas().set_index('CIDint')
BBC_mask = (df_BBC_ORIGIN["HOST_NMATCH"] > 0) & (df_BBC_ORIGIN['zHD'].between(zrange[0], zrange[1]))
df_BBC_ORIGIN = df_BBC_ORIGIN[BBC_mask]

CIDarr = df_BBC_ORIGIN.index.values
for k in models:
    print(k)
    FIT_FILE = FIT_DIR /  f'LSST_FIT_LSST_{k}/output/PIP_LSST_UCHUU_MOCK00_P21_VARIATIONS_BC_LSST_{k}/FITOPT000.FITRES.gz'
    df_STDFIT = ascii.read(FIT_FILE).to_pandas().set_index('CID')
    df_STDFIT[df_STDFIT.apply(pf.positive_def, axis=1)]
    CIDarr = np.intersect1d(CIDarr, df_STDFIT.index)

print(f'PROP: {len(CIDarr) / len(df_BBC_ORIGIN.index.values):.3f}')

del df_BBC_ORIGIN
del df_STDFIT

In [ ]:
# Open files
data_fit = {}
par_fit = {}
resfit = pd.read_csv('../results_p21_var/RES_MOCK00_P21_VAR_zrange_0.02_0.1_nojax.csv')

for k in models:
    # STANDARD DATA
    FIT_FILE = FIT_DIR /  f'LSST_FIT_LSST_{k}/output/PIP_LSST_UCHUU_MOCK00_P21_VARIATIONS_BC_LSST_{k}/FITOPT000.FITRES.gz'
    df_STDFIT = ascii.read(FIT_FILE).to_pandas().set_index('CID').loc[CIDarr]
    
    df_STDFIT = df_STDFIT.apply(pf.x0_to_mB_err, axis=1)
        
    data_fit[k] = flip.data_vector.snia_vectors.VelFromSALTfit({
        "ra": np.deg2rad(df_STDFIT['HOST_RA'].values),
        "dec": np.deg2rad(df_STDFIT['HOST_DEC'].values),
        "mb": df_STDFIT['mB'].values ,
        "x1": df_STDFIT["x1"].values,
        "c": df_STDFIT["c"].values,
        "e_mb": df_STDFIT["mBERR"].values,
        "e_x1": df_STDFIT["x1ERR"].values,
        "e_c": df_STDFIT["cERR"].values,
        "cov_mb_x1": df_STDFIT["COV_mB_x1"].values,
        "cov_mb_c": df_STDFIT["COV_mB_c"].values,
        "cov_x1_c": df_STDFIT["COV_x1_c"].values,
        "zobs": df_STDFIT["zCMB"].values,
        "rcom_zobs": cosmo.comoving_distance(df_STDFIT["zCMB"].values).value * cosmo.h,
        "hubble_norm": 100 * cosmo.efunc(df_STDFIT["zCMB"].values),
        "host_logmass": df_STDFIT["HOST_LOGMASS"].values,
        "vtrue": df_STDFIT['SIM_HOSTLIB_VPEC'].values,
        }, 
        vel_estimator="full", 
        h=cosmo.h)
    

In [ ]:
resfit.set_index('model', inplace=True)

In [ ]:
xrange = (-5, 5)
plt.hist(dmu_p21 / (1.48 * np.median(np.abs(dmu_p21))), range=xrange,bins=61, density=True)
plt.plot(xth, scs.norm().pdf(xth), c='k', alpha=0.6)
plt.axvline(0, ls='--', c='k');

In [ ]:
fig, ax = plt.subplots(3,3,dpi=150, sharex=True, sharey=True, figsize=(10, 10))
fig2, ax2 = plt.subplots(3,3,dpi=150, sharex=True, sharey=True, figsize=(10, 10))

xrange = (-0.5, 0.5)
xrange2 = (-5, 5)

nbins=41
dmu_p21 = data_fit['P21'].compute_distance_modulus_difference({t: resfit.loc['P21'][t + '_STDFIT'] for t in TRIPP_KEYS})
dmu_err_p21 = np.sqrt(data_fit['P21'].compute_observed_distance_modulus_variance({t: resfit.loc['P21'][t + '_STDFIT'] for t in TRIPP_KEYS}))
i = 0
for k in data_fit:
    if k =='P21':
        continue
    else:
        res = {t: resfit.loc[k][t + '_STDFIT'] for t in TRIPP_KEYS}
        dmu = data_fit[k].compute_distance_modulus_difference(res)
        dmu_err = np.sqrt(data_fit[k].compute_observed_distance_modulus_variance(res))
        
        ax[i // 3, i % 3].hist(dmu_p21, range=xrange, bins=nbins, histtype='step', lw=1.5)
        ax[i // 3, i % 3].hist(dmu, range=xrange, bins=nbins, histtype='step', lw=1.5, label=k)
        ax[i // 3, i % 3].legend(loc='upper right',fontsize=5)

        ax2[i // 3, i % 3].hist(dmu / (1.48 * np.median(np.abs(dmu))), range=xrange2, bins=nbins, histtype='step', lw=1.5, label=k, density=True)
        ax2[i // 3, i % 3].hist(dmu / dmu_err, range=xrange2, bins=nbins, histtype='step', lw=1.5, label=k, density=True)
        ax2[i // 3, i % 3].plot(xth, scs.norm().pdf(xth), c='k', alpha=0.6)
        ax2[i // 3, i % 3].legend(loc='upper right',fontsize=5)
        ax2[i // 3, i % 3].set_yscale('log')
        i += 1
xth = np.linspace(-5, 5, 1000)


In [ ]:
fig, ax = plt.subplots(3,3,dpi=150, sharex=True, sharey=True, figsize=(10, 10))
fig2, ax2 = plt.subplots(3,3,dpi=150, sharex=True, sharey=True, figsize=(10, 10))

xrange = (-4500, 4500)
xrange2 = (-5, 5)

nbins=51
v_p21, vvar_p21 = data_fit['P21']({t: resfit.loc['P21'][t + '_STDFIT'] for t in TRIPP_KEYS})
dv_p21 = v_p21 - data_fit['P21'].data['vtrue']
verr_p21 = np.sqrt(vvar_p21)
i = 0
for k in data_fit:
    if k =='P21':
        continue
    else:
        res = {t: resfit.loc[k][t + '_STDFIT'] for t in TRIPP_KEYS}
        v, vvar = data_fit[k](res)
        verr = np.sqrt(vvar)
        dv = v - data_fit[k].data['vtrue']
        
        ax[i // 3, i % 3].hist(dv_p21, range=xrange, bins=nbins, histtype='step', lw=1.5)
        ax[i // 3, i % 3].hist(dv, range=xrange, bins=nbins, histtype='step', lw=1.5, label=k)
        ax[i // 3, i % 3].legend(loc='upper right',fontsize=5)
        
        ax2[i // 3, i % 3].hist(dv_p21 / (1.48 * np.median(np.abs(dv_p21))), range=xrange2, bins=nbins, histtype='step', lw=1.5, density=True)
        ax2[i // 3, i % 3].hist(dv / (1.48 * np.median(np.abs(dv))), range=xrange2, bins=nbins, histtype='step', lw=1.5, label=k, density=True)
        ax2[i // 3, i % 3].plot(xth, scs.norm().pdf(xth), c='k', alpha=0.6)
        ax2[i // 3, i % 3].legend(loc='upper right',fontsize=5)
        ax2[i // 3, i % 3].set_yscale('log')
        i += 1
xth = np.linspace(-5, 5, 1000)


In [ ]:
fig, ax = plt.subplots(dpi=150)
fig2, ax2 = plt.subplots(dpi=150)

xrange = (-4500, 4500)
xrange2 = (-5, 5)

nbins=41

for k in data_fit:
    res = {t: resfit.loc[k][t + '_STDFIT'] for t in TRIPP_KEYS}
    v, vvar = data_fit[k](res)
    verr = np.sqrt(vvar)
    dv = v - data_fit[k].data['vtrue']
    ax.hist(dv, range=xrange, bins=nbins, histtype='step', lw=1.5, label=k)
    ax2.hist(dv / (1.48 * np.median(np.abs(dv))), range=xrange2, bins=nbins, histtype='step', lw=1.5, label=k, density=True)

    #ax2.hist(dv / verr, range=xrange2, bins=nbins, histtype='step', lw=1.5, label=k, density=True)
xth = np.linspace(-5, 5, 1000)
ax2.plot(xth, scs.norm().pdf(xth), c='k', alpha=0.6)
ax.legend()


In [ ]:
fig, ax = plt.subplots(dpi=150)

xrange = (-4500, 4500)

nbins=41

for k in data_fit:
    ax.hist(dv, range=xrange, bins=nbins, histtype='step', lw=1.5, label=k)

    #ax2.hist(dv / verr, range=xrange2, bins=nbins, histtype='step', lw=1.5, label=k, density=True)
xth = np.linspace(-5, 5, 1000)
ax2.plot(xth, scs.norm().pdf(xth), c='k', alpha=0.6)
ax.legend()